### 단어를 시퀀스로 처리하기: 시퀀스 모델 방식

#### 첫 번째 예제

**데이터 다운로드**

In [1]:
!rm -r aclImdb
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!rm -r aclImdb/train/unsup

rm: cannot remove 'aclImdb': No such file or directory
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  21.1M      0  0:00:03  0:00:03 --:--:-- 21.1M


**데이터 준비**

In [2]:
import os, pathlib, shutil, random
from tensorflow import keras
batch_size = 32
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"

for category in ("neg", "pos"):
    os.makedirs(val_dir / category)
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)
    num_val_samples = int(0.2 * len(files))
    val_files = files[-num_val_samples:]
    for fname in val_files:
        shutil.move(train_dir / category / fname,
                    val_dir / category / fname)

train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)
text_only_train_ds = train_ds.map(lambda x, y: x)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


**정수 시퀀스 데이터셋 준비하기**

In [3]:
from tensorflow.keras import layers

max_length = 600
max_tokens = 20000
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)
text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

**원-핫 인코딩된 벡터 시퀀스로 시퀀스 모델 만들기**

### Bidirectional LSTM (Long Short-Term Memory)
- 순환 신경망(RNN, Recurrent Neural Network)의 한 종류로서, 시퀀스 또는 시계열 데이터를 처리하는데 사용. LSTM은 RNN의 단점 중 하나인 장기 종속성 문제를 해결하는 방법을 제공.

- Bidirectional LSTM은 기본적인 LSTM 구조에 양방향성이 추가된 형태이며 시퀀스를 두 방향(과거에서 미래로, 미래에서 과거로)에서 동시에 처리하므로 양쪽 방향의 정보를 모두 활용할 수 있다.

- Bidirectional LSTM 특징

  - 양방향 정보 처리: Bidirectional LSTM은 입력 시퀀스를 두 방향으로 읽는데 이는 미래의 입력이 현재의 출력에 영향을 줄 수 있음을 의미. 이러한 특성 때문에, Bidirectional LSTM은 문맥이 양쪽 방향에 걸쳐 있는 자연어 처리(NLP) 작업에 특히 유용하여, 기계 번역, 질문 응답 시스템, 감성 분석 등에 주로 사용.

  - 내부 메모리: Bidirectional LSTM은 LSTM의 기본적인 특성인 "장기 기억"을 가지고 있어서 과거의 정보를 기억하고, 이를 필요에 따라 사용하거나 잊어버릴 수 있다. 이러한 특성은 시퀀스 내에서 장기적인 종속성을 학습하는 데 효과적.

  - 복잡한 구조: Bidirectional LSTM은 두 개의 LSTM 레이어가 역방향으로 연결되어 있으므로, 일반적인 LSTM보다 매개변수가 더 많아서, 더 많은 계산 자원을 필요로 하고, 과적합을 방지하기 위한 주의가 필요.

TensorFlow에서는 tensorflow.keras.layers.Bidirectional 레이어를 사용하여 Bidirectional LSTM을 쉽게 구현할 수 있으며 이 레이어는 일반 LSTM 레이어를 인자로 받아 양방향 LSTM을 구현.

In [5]:
#  depth=max_tokens는 one-hot 벡터의 차원 수를 max_tokens로 설정하라는 것을 의미

import tensorflow as tf
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = tf.one_hot(inputs, depth=max_tokens) # tf.one_hot 함수를 사용하여 입력 텐서를 one-hot 벡터로 변환. depth=max_tokens는 one-hot 벡터의 차원 수를 max_tokens로 설정하라는 것을 의미
x = layers.Bidirectional(layers.LSTM(32))(embedded) # layers.Bidirectional 함수를 사용하여 양방향 LSTM 레이어를 생성.
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 tf.one_hot_1 (TFOpLambda)   (None, None, 20000)       0         
                                                                 
 bidirectional (Bidirectiona  (None, 64)               5128448   
 l)                                                              
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 5,128,513
Trainable params: 5,128,513
Non-trainable params: 0
___________________________________________________

**첫 번째 시퀀스 모델 훈련하기**

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint("one_hot_bidir_lstm.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("one_hot_bidir_lstm.keras")
print(f"테스트 정확도: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10
625/625 [==============================] - 174s 261ms/step - loss: 0.5583 - accuracy: 0.7185 - val_loss: 0.3906 - val_accuracy: 0.8468
Epoch 2/10
625/625 [==============================] - 162s 258ms/step - loss: 0.3535 - accuracy: 0.8665 - val_loss: 0.3640 - val_accuracy: 0.8548
Epoch 3/10
625/625 [==============================] - 163s 261ms/step - loss: 0.2866 - accuracy: 0.8972 - val_loss: 0.3278 - val_accuracy: 0.8666
Epoch 4/10
625/625 [==============================] - 163s 261ms/step - loss: 0.2407 - accuracy: 0.9154 - val_loss: 0.3592 - val_accuracy: 0.8752
Epoch 5/10
625/625 [==============================] - 163s 261ms/step - loss: 0.2163 - accuracy: 0.9262 - val_loss: 0.3046 - val_accuracy: 0.8748
Epoch 6/10
625/625 [==============================] - 163s 260ms/step - loss: 0.1882 - accuracy: 0.9365 - val_loss: 0.3631 - val_accuracy: 0.8854
Epoch 7/10
625/625 [==============================] - 163s 260ms/step - loss: 0.1634 - accuracy: 0.9464 - val_loss: 0.3603 -

#### 단어 임베딩 이해하기

#### 임베딩 층으로 단어 임베딩 학습하기

**`Embedding` 층 만들기**

In [ ]:
embedding_layer = layers.Embedding(input_dim=max_tokens, output_dim=256)

**밑바닥부터 훈련하는 `Embedding` 층을 사용한 모델**

In [ ]:
# 임베딩 층과 양방향 LSTM을 가진 딥러닝 모델을 생성하고, 이를 훈련하고 평가하는 과정

inputs = keras.Input(shape=(None,), dtype="int64") # shape=(None,)는 입력 시퀀스의 길이가 가변적임을 의미
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256)(inputs) # 임베딩 층을 정의하고 입력에 적용
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_lstm.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("embeddings_bidir_lstm.keras")
print(f"테스트 정확도: {model.evaluate(int_test_ds)[1]:.3f}")

#### 패딩과 마스킹 이해하기

**마스킹을 활성화한 `Embedding` 층 사용하기**

In [ ]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(
    input_dim=max_tokens, output_dim=256, mask_zero=True)(inputs) # mask_zero=True는 0의 입력을 패딩으로 간주하고 처리 과정에서 무시
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_lstm_with_masking.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("embeddings_bidir_lstm_with_masking.keras")
print(f"테스트 정확도: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 256)         5120000   
                                                                 
 bidirectional (Bidirectiona  (None, 64)               73984     
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 5,194,049
Trainable params: 5,194,049
Non-trainable params: 0
___________________________________________________

#### 사전 훈련된 단어 임베딩 사용하기

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2023-07-13 03:24:37--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-07-13 03:24:38--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-07-13 03:24:38--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

**GloVe 단어 임베딩 파일 파싱하기**

In [ ]:
# GloVe (Global Vectors for Word Representation) 단어 임베딩을 읽고 파싱하는 과정
import numpy as np
path_to_glove_file = "glove.6B.100d.txt"

embeddings_index = {} # GloVe에서 읽어들인 단어 임베딩을 저장할 딕셔너리를 초기화
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1) # 각 줄을 최대 1번 분할하여 단어와 임베딩 벡터를 분리
        coefs = np.fromstring(coefs, "f", sep=" ") # 임베딩 벡터를 문자열에서 실수로 구성된 NumPy 배열로 변환. 각 원소는 공백으로 구분
        embeddings_index[word] = coefs # 단어와 해당 임베딩 벡터를 딕셔너리에 저장

print(f"단어 벡터 개수: {len(embeddings_index)}")

단어 벡터 개수: 400000


**GloVe 단어 임베딩 행렬 준비하기**

In [ ]:
#  텍스트 데이터에 맞는 임베딩 행렬을 만드는 과정

embedding_dim = 100

vocabulary = text_vectorization.get_vocabulary() # text_vectorization에서 사용한 단어장(vocabulary)을 불러온다.
word_index = dict(zip(vocabulary, range(len(vocabulary)))) # 단어장의 각 단어에 고유한 정수 인덱스를 할당합니다. 이를 통해 각 단어를 해당 단어의 인덱스로 변환

embedding_matrix = np.zeros((max_tokens, embedding_dim)) # 텍스트 데이터에 맞게 변환될 임베딩 행렬을 초기화
for word, i in word_index.items():
    if i < max_tokens:
        embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector # 임베딩 행렬의 i번째 행에 해당 단어의 임베딩 벡터를 설정합니다. 이를 통해 우리가 가진 텍스트 데이터의 각 단어에 GloVe 임베딩 벡터가 매핑

In [ ]:
# 텍스트를 임베딩 벡터로 변환하는 케라스 임베딩 레이어를 초기화
embedding_layer = layers.Embedding(
    max_tokens, # 텍스트 데이터의 고유한 단어 수
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix), # 임베딩 레이어의 가중치 초기화 방식을 정의
    trainable=False,
    mask_zero=True, # 패딩된 0 값에 대해 마스크를 적용하여 실제 계산에서 제외
)

**사전 훈련된 임베딩을 사용하는 모델**

In [ ]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = embedding_layer(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("glove_embeddings_sequence_model.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("glove_embeddings_sequence_model.keras")
print(f"테스트 정확도: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_1 (Embedding)     (None, None, 100)         2000000   
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               34048     
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 2,034,113
Trainable params: 34,113
Non-trainable params: 2,000,000
____________________________________________